In [9]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import mediapipe as mp 
from alert import email_alert

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# define a helper function to detected face
def detect_and_predict_blood(frame, bloodNet, check):
    # This function takes in an input of the frame, the model, and a "check" variable for debugging purposes
    # You can remove this check parameter without breaking the code, it was for helping me discover what each variable was easily

    detection_list = []
    preds = []

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = img_to_array(frame)
    frame = preprocess_input(frame)

    detection_list.append(frame)

    if check == True:
        print("detection_list before turning into np.array:")
        #print(type(detection_list))
        #print(detection_list)
    # Create a numpy array to apply the model, in case that we have multiple things (subject to change depending on if we change the input)
    detection_list = np.array(detection_list, dtype="float32")
    preds = bloodNet.predict(detection_list, batch_size=32) # DV: bloodNet should be our model to detect blood in a frame
        
    # Return an array on whether there is blood or no blood in the frame:
    # Example: array([[0.00736171 0.99263835]], dtype="float32"). When you access this value, you'll get [[0.00736171 0.99263835]]
    return preds

# Load the blood detector model from disk
bloodNet = load_model('./blood_noblood_classifier.model') # DV: Change the path to our model

alert_sent = False
blood_counter = 0

# Initialize the video stream and allow the camera sensor to warm up
#vs = VideoStream(src=0).start()

cap = cv2.VideoCapture(0)

#time.sleep(2.0)

print("hands")
hands = mp_hands.Hands(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)
print("hands2")


# Loop over the frames from the video stream
while cap.isOpened():
    # Grab the frame from the threaded video stream and resize it to have a maximum width of 400 pixels
    
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue
        
    frame = imutils.resize(frame, width=400)

    results = hands.process(frame)

    
    hand = False
    
    if results.multi_hand_landmarks:
        hand = True
    
    if hand == True:
        # Run the prediction on blood on the entire frame
        preds = detect_and_predict_blood(frame, bloodNet, False)

        # Obtain the prediction. preds is an array right now that looks like, for example: [[0.00736171 0.99263835]]
        # So we have to create a tuple from the values inside
        (noblood, blood) = (preds[0][0], preds[0][1])
    else:
        (noblood, blood) = (100,0)
        
    
    # Assign a label on whether blood is detected on the screen and assign a color
    label = "Blood" if blood > noblood else "no Blood"
    color = (0, 0, 255) if label == "Blood" else (0, 255, 0)

    # Include the probability in the label
    label = "{}: {:.2f}%".format(label, max(blood, noblood) * 100)
    if alert_sent is True:
        label = "Alert has been sent! Please hold on for help!"
        color = (0, 0, 255)

    # Display the label on the frame
    cv2.putText(frame, label, (20, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)

    # Show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If blood is detected and an alert has not been sent, send an alert only once after a certain number of frames have passed in a row
    if hand == True and blood > noblood and alert_sent is False:
        blood_counter = blood_counter + 1
        # If the blood counts up for a certain amount of frames, send the alert only one time
        # 30 frames is approximately 2 seconds?
        if blood_counter > 30:  # This is the number of frames
            email_alert("red alert test", "some one is dyinggggg help", "9168376779@messaging.sprintpcs.com")
            alert_sent = True
    # Else, reset the counter
    elif hand == True and noblood < blood and alert_sent is False:
        blood_counter = 0

    if key == ord("w"):
        alert_sent = False
        blood_counter = 0


    # If the q is pressed, break from the loop
    if key == ord("q"):
        # DV: Check the last frame
        # Debug/discovery stuff
        # print("Q was pressed")
        # preds = detect_and_predict_blood(frame, bloodNet, True)
        # print("Print preds")
        # print(preds)
        # (blood, noblood) = (preds[0][0], preds[0][1])
        # print("The blood no blood tuple")
        # print((blood, noblood))
        # if blood > noblood:
        #     print("Blood detected")
        # else:
        #     print("No blood detected")

        break
        
# Do a bit of cleanup
hands.close()
cap.release()

cv2.destroyAllWindows()

hands
hands2


In [4]:
import time
print("Hello World")
time.sleep(2.0)
print("hi")

Hello World
hi


In [ ]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import mediapipe as mp
from alert import email_alert

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# define a helper function to detected face
def detect_and_predict_blood(frame, bloodNet, check):
    # This function takes in an input of the frame, the model, and a "check" variable for debugging purposes
    # You can remove this check parameter without breaking the code, it was for helping me discover what each variable was easily

    detection_list = []
    preds = []

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = img_to_array(frame)
    frame = preprocess_input(frame)

    detection_list.append(frame)

    if check == True:
        print("detection_list before turning into np.array:")
        #print(type(detection_list))
        #print(detection_list)
    # Create a numpy array to apply the model, in case that we have multiple things (subject to change depending on if we change the input)
    detection_list = np.array(detection_list, dtype="float32")
    preds = bloodNet.predict(detection_list, batch_size=32) # DV: bloodNet should be our model to detect blood in a frame
        
    # Return an array on whether there is blood or no blood in the frame:
    # Example: array([[0.00736171 0.99263835]], dtype="float32"). When you access this value, you'll get [[0.00736171 0.99263835]]
    return preds

# Load the blood detector model from disk
bloodNet = load_model('./blood_noblood_classifier.model') # DV: Change the path to our model

alert_sent = False
blood_counter = 0

# Initialize the video stream and allow the camera sensor to warm up
vs = VideoStream(src=0).start()
time.sleep(2.0)

hands = mp_hands.Hands(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Loop over the frames from the video stream
while True:
    
    # Grab the frame from the threaded video stream and resize it to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    print(type(image))
    results = hands.process(image)
    
    # Run the prediction on blood on the entire frame
    preds = detect_and_predict_blood(frame, bloodNet, False)

    # Obtain the prediction. preds is an array right now that looks like, for example: [[0.00736171 0.99263835]]
    # So we have to create a tuple from the values inside
    (noblood, blood) = (preds[0][0], preds[0][1])
    
    # Assign a label on whether blood is detected on the screen and assign a color
    label = "Blood" if blood > noblood else "No Blood"
    color = (0, 0, 255) if label == "Blood" else (0, 255, 0)
    
    # Include the probability in the label
    label = "{}: {:.2f}%".format(label, max(blood, noblood) * 100)
    if alert_sent is True:
        label = "Alert has been sent! Please hold on for help!"
        color = (0, 0, 255)
    
    # Display the label on the frame
    cv2.putText(frame, label, (20, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)

    # Show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    # If blood is detected and an alert has not been sent, send an alert only once after a certain number of frames have passed in a row
    if blood > noblood and alert_sent is False:
        blood_counter = blood_counter + 1
        # If the blood counts up for a certain amount of frames, send the alert only one time
        # 30 frames is approximately 2 seconds?
        if blood_counter > 30:  # This is the number of frames
            email_alert("red alert test", "some one is dyinggggg help", "9168376779@messaging.sprintpcs.com")
            alert_sent = True
    # Else, reset the counter
    elif noblood < blood and alert_sent is False:
        blood_counter = 0
    
    if key == ord("w"):
        alert_sent = False
        blood_counter = 0


    # If the q is pressed, break from the loop
    if key == ord("q"):
        # DV: Check the last frame
        # Debug/discovery stuff
        # print("Q was pressed")
        # preds = detect_and_predict_blood(frame, bloodNet, True)
        # print("Print preds")
        # print(preds)
        # (blood, noblood) = (preds[0][0], preds[0][1])
        # print("The blood no blood tuple")
        # print((blood, noblood))
        # if blood > noblood:
        #     print("Blood detected")
        # else:
        #     print("No blood detected")

        break
        
# Do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

In [2]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import mediapipe as mp
from alert import email_alert

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)

hands.process?


In [6]:
!pip install mediapipe

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\dylan vu\\visual studio code projects\\virtualenvs\\python37\\lib\\site-packages\\~umpy\\.libs\\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Users\Dylan Vu\Visual Studio Code Projects\virtualenvs\python37\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached mediapipe-0.8.3-cp37-cp37m-win_amd64.whl (83.0 MB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)


You should consider upgrading via the 'C:\Users\Dylan Vu\Visual Studio Code Projects\virtualenvs\python37\Scripts\python.exe -m pip install --upgrade pip' command.
